# Jane Slagle
# CS 138 RL - Programming Assignment 3
# FrozenLake.ipynb

In [1]:
import gym
from gymnasium.envs.toy_text.frozen_lake import generate_random_map
import numpy as np

## Set up the frozen lake environment from gym using a randomly generated grid to respresent the lake as well as perform various other set up tasks including getting the number of states, actions out and initializing all needed parameters for running through the algorithms:

In [38]:
#generate a random 4x4 grid map to create frozen lake env with
rand_map = generate_random_map(size=4)

#set up frozen lake env from gym
env = gym.make("FrozenLake-v1", desc=rand_map, is_slippery = False)

#print out what 4x4 frozen lake grid env looks like
for row in rand_map:
    print(row)

SFFH
FFFF
FFFF
FFFG


In [6]:
#get number states, actions out of env
num_states = env.observation_space.n
num_acts = env.action_space.n

print("num states: " + str(num_states))
print("num actions: " + str(num_acts))

num states: 16
num actions: 4


In [7]:
#need gamma for algorithm equations -- use 0.9 as gamma bc common value for it based off recorded lectures
gamma = 0.9 

#define all params will need for when actually run through algorithms
num_eps = 5000        #num training episodes
max_steps = 100       #max steps per episode, if dont have then never reach goal state when run algors

#init val funcs V for 2 algor (7.13 + 7.2 and 7.1 + 7.9) are running here
V_algor_1 = np.zeros(num_states)  #let equations 7.13 + 7.2 be rep as algor_1
V_algor_2 = np.zeros(num_states)  #let equations 7.1 + 7.9 be rep as algor_2

## Define helper functions for executing the 2 algorithms, including generating data from each episode, specifying the behavior and target policies for the algorithms, and calculating the importance sampling ratio with those policies required for the algorithm equations:

In [8]:
#generate tuple of info for each episode using inputted given policy
def gen_eps(env, policy, max_steps):
    #each eps = tuple of (state, act, reward, next state, done) add done to each eps tuple bc will want
    #to use it to get results from running algorithm at end
    all_eps = []            #store all data from each episode
    state = env.reset()     #make sure the env is reset for each episode
    
    #limit each episode run to number of max steps
    for _ in range(max_steps):
        act = policy()      #simluate taking action in each episode
        
        #now get info out from taking action in episode!
        #env.step method takes act just took + returns the next state, reward, if done + other info
        next_state, reward, done, info = env.step(act)
        all_eps.append((state, act, reward, next_state, done))  #add this episode to list of all episodes
        
        state = next_state  #update state for next step take
        
        #use done value just got out to see if have reached terminal state. if have, then exit bc 
        #means no more episodes to generate data for!
        if done:
            break
            
    return all_eps  #return info from episodes just found!

In [9]:
#both algor have rho in them = importance sampling ratio (eq 7.10) so define that!
#need behavior, target policies for calcuating importance sampling ratio!

#make behavior policy random policy, selects action to take by randomly choosing from all actions have
def b_policy():
    return np.random.choice(num_acts)

#make target policy greedy policy, will select action to take by choosing one w/ best Q func act val 
def t_policy(state, V):
    Q = []    #store all Q act values
    
    #loop through all actions have + find all act vals for each state
    for act in range(num_acts):
        env.reset()   #make sure env is reset for each action
        
        #simulate env step, get all info want out of it
        next_state, reward, done, info = env.step(act)
        
        #calc Q act val where Q formula = immed. reward + gamma * future reward where future reward is
        #from the value func V (V is inputted in func, woohoo!)
        q_val = reward + (gamma * V[next_state])
        Q.append(q_val)
        
    #return act w/ highest expected Q val (bc greedy approach)
    return np.argmax(Q)

#now use those policy funcs here! = exactly equation 7.10 given in book
def import_samp_ratio(eps, t_policy, b_policy, V):
    #store all pi/b ratios for each time step in eps so that can take product of them all at end
    rho = []
    
    #loop over each (state, act) pair in the eps
    #eps returns (state, act, reward, next state, done state) tuple but only care about state, act here
    for state, act, reward, next_state, done in eps:
        #for each (state, act) pair in eps, need calc rho val
        #pi = prob take act a under target policy at state s, so 1 if acts are same, else 0
        if act == t_policy(state, V):
            pi = 1.0
        else:
            pi = 0.0
            
        #b = 1/num acts for uniform rand behavior policy
        b = 1.0 / num_acts
        
        rho.append(pi / b)
        
    #product runs over all time steps in eps
    return np.prod(rho)

## Simulate running through algorithm 1 (equations 7.13 + 7.2):

In [10]:
#algor 1: equations 7.13 + 7.2 from book - corresponds with V_algor_1

def run_algor_1(env, num_eps, gamma):
    reached_goal_algor_1 = 0      #keep count of number times agent reached goal state on frozen lake grid
    didnt_reach_goal_algor_1 = 0  #keep count number times agent didnt reach goal state, from either just
                                  #not reaching goal state or from falling in hole
    steps_taken_algor_1 = []      #keep track num steps took to reach goal state, will store num 
                                  #steps for each episode
                
    #run through for all episodes have
    for _ in range(num_eps):
        #actually generate episode for each episode loop through!
        #use behavior policy to generate data for each episode
        eps = gen_eps(env, b_policy, max_steps) 
        
        #find G in equation 7.13, store as array bc compute G_t:h based on G_t+1:h which means need
        #compute next value before compute current one and if store as array then can do this!
        G = np.zeros(len(eps))
        
        #find equation 7.13 1st bc it calcs G and need G for equation 7.2
        #need next step to find current step so need work through time steps bwards
        #specifically: need start at end of episode + work bwards from there, from 2nd to last time step
        #bc at last time step, there = no next time step after it
        for t in range(len(eps) -2, -1, -1):
            #get importance sampling ratio value out, want for current time step t's eps
            rho_t = import_samp_ratio([eps[t]], t_policy, b_policy, V_algor_1)
            
            #get all of info for specific episode time step out so that can use it!!!
            state, act, reward, next_state, done = eps[t]
            
            #equation 7.13:
            G[t] = rho_t * (reward + gamma * G[t+1]) + (1 - rho_t) * V_algor_1[state]
            
        #G vals computed now, so can use them to update val func w/ equation 7.2!!!
        for t in range(len(eps)):
            #get all info out of eps
            state, act, reward, next_state, done = eps[t]
            
            #update val func w/ equation 7.2
            V_algor_1[state] += gamma * (G[t] - V_algor_1[state])
            
            #check if in terminal state (if eps over)
            #if eps is over, check if it reached goal or not
            if done:
                if reward == 1:
                    #then means reached goal state - woohoo!
                    reached_goal_algor_1 += 1
                    steps_taken_algor_1.append(t+1) #keep track of steps taken to reach goal so that 
                                                    #can get their count in results
                elif reward == 0:
                    #then means either didnt reach goal or fell in hole... boooooo
                    didnt_reach_goal_algor_1 += 1

                break #eps ends when are in terminal state so break
    
    #get results from running algor out
    get_results(steps_taken_algor_1, reached_goal_algor_1, didnt_reach_goal_algor_1)

## Simulate running through algorithm 2 (equations 7.9 + 7.1):

In [11]:
#algor 2: equations 7.9 + 7.1 from book - corresponds with V_algor_2

def run_algor_2(env, num_eps, gamma):
    reached_goal_algor_2 = 0      #keep count of number times agent reached goal state on frozen lake grid
    didnt_reach_goal_algor_2 = 0  #keep count number times agent didnt reach goal state, from either just
                                  #not reaching goal state or from falling in hole
    steps_taken_algor_2 = []      #keep track num steps took to reach goal state, will store num 
                                  #steps for each episode
        
    #run through for all episodes have
    for _ in range(num_eps):
        #actually generate episode for each episode loop through!
        #use behavior policy to generate data for each episode
        eps = gen_eps(env, b_policy, max_steps) 
        
        #loop through all time steps for each indiv eps and simulate agent navigating over frozen lake!!!
        for t in range(len(eps)):
            #get all of the info for specific episode time step out so that can use it!!!
            state, act, reward, next_state, done = eps[t]
            
            #compute G = equation 7.1 from book AHHHH!!!
            #find 7.1 before 7.9 bc use 7.1 in 7.9
            G = 0
            #start from current time step t, run through until reach end of episode
            for k in range(t, len(eps)):
                G += gamma ** (k - t) * reward 
                
            #get importance sampling ratio so that can use it in equation 7.9
            rho_t = import_samp_ratio([eps[t]], t_policy, b_policy, V_algor_2)
            
            #update val func using equation 7.9 from book + using G just found w/ equation 7.1
            V_algor_2[state] += rho_t * (G - V_algor_2[state])
            
            #check if in terminal state (if eps over)
            #if eps is over, check if it reached goal or not
            if done:
                if reward == 1:
                    #then means reached goal state - woohoo!
                    reached_goal_algor_2 += 1
                    steps_taken_algor_2.append(t+1) #keep track of steps taken to reach goal so that 
                                                    #can get their count in results
                elif reward == 0:
                    #then means either didnt reach goal or fell in hole... boooooo
                    didnt_reach_goal_algor_2 += 1

                break #eps ends when are in terminal state so break
       
    #get results from running algor out
    get_results(steps_taken_algor_2, reached_goal_algor_2, didnt_reach_goal_algor_2)

## Get the results out from running the 2 algorithms:

In [12]:
#print out results want: avg num steps took to reach goal state, percentage time reached goal state,
#percentage time fell in hole or didnt reach goal state when simulating across all episodes

def get_results(steps_taken, goal_count, not_goal_count):
    #take mean of steps_taken bc it is an array for each episode so get average of num steps taken across
    #all episodes
    avg_steps_taken = np.mean(steps_taken)
    print("The goal state was reached in an average of " + str(avg_steps_taken) + " steps.")
    
    #get percentage of number times goal state was reached out of all episodes
    goal_reached_percent = (goal_count / num_eps) * 100
    print("The goal state was reached " + str(goal_reached_percent) + " percent of the time.")
    
    #get percentage of number times goal state was not reached out of all episodes
    goal_not_reached_percent = (not_goal_count / num_eps) * 100
    print("The goal state was not reached " + str(goal_not_reached_percent) + " percent of the time, either from ending the episode in a frozen state or from falling into a hole.")

## Results from running the first algorithm (7.13 + 7.2):

In [39]:
#run advanced algorithm to get results
run_algor_1(env, num_eps, gamma)

The goal state was reached in an average of 30.657945736434108 steps.
The goal state was reached 41.28 percent of the time.
The goal state was not reached 58.720000000000006 percent of the time, either from ending the episode in a frozen state or from falling into a hole.


## Results from running the second algorithm (7.9 + 7.1):

In [40]:
#run simple algorithm to get results
run_algor_2(env, num_eps, gamma)

The goal state was reached in an average of 30.71672828096118 steps.
The goal state was reached 43.28 percent of the time.
The goal state was not reached 56.720000000000006 percent of the time, either from ending the episode in a frozen state or from falling into a hole.


## Additional Question - running the same frozen lake environment with SARSA on-policy TD control algorithm:

In [63]:
#SARSA on-policy TD control algorithm from Section 6.4, page 130 in book
#exactly follow the pseudocode given in book!!!

#normal values for both wanted params with algorithm according to recorded lectures
alph = 0.1
epsilon = 0.1   

def run_sarsa_algor(env, num_eps, max_steps, gamma, alph, epsilon):
    Q_sarsa = np.zeros((num_states, num_acts))  #init Q(s,a) for all states, actions
    
    reached_goal_sarsa = 0      #keep count of number times agent reached goal state on frozen lake grid
    didnt_reach_goal_sarsa = 0  #keep count number times agent didnt reach goal state, from either just
                                #not reaching goal state or from falling in hole
    steps_taken_sarsa = []      #keep track num steps took to reach goal state, will store num 
                                #steps for each episode
        
    #loop over all episodes have
    for _ in range(num_eps):
        state = env.reset()     #init state S
        
        #choose action A from state S using eps-greedy policy derived from Q_sarsa
        if np.random.uniform(0,1) < epsilon:  
            #means want explore, can call b_policy() func wrote for this!
            act = b_policy()
        else:
            #means want exploit, choose the act w/ max Q func val here
            act = np.argmax(Q_sarsa[state])
            
        #loop for each step of episode (each episode is at most max_steps amount long):
        for t in range(max_steps):
            #take action A, observation R, next state S'
            next_state, reward, done, info = env.step(act)
            
            #choose next action from next state using policy derived from Q_sarsa (eps-greedy)
            if np.random.uniform(0,1) < epsilon:
                #mans explore, call b_policy() func wrote for tihs!
                next_act = b_policy()
            else:
                #means want exploit, choose act w/ max Q func val here
                next_act = np.argmax(Q_sarsa[next_state])
            
            #update Q(s,a) using formula given in book
            #Q(s,a) = Q(s,a) + alpha[R + gamma Q(next state, next act) - Q(s,a)]
            Q_sarsa[state, act] += alph * (reward + (gamma * Q_sarsa[next_state, next_act] - Q_sarsa[state, act]))
            
            #update S + update A for next run through loop
            #S = next state, A = next action
            state = next_state
            act = next_act
            
            #keep doing this until the state is terminal so check if the state is or not!
            #then figure out if reached goal or not for getting results out
            if done:
                if reward == 1:
                    #then means reached goal state - woohoo!
                    reached_goal_sarsa += 1
                    steps_taken_sarsa.append(t+1) #keep track of steps taken to reach goal so that can 
                                                  #get their count in results
                elif reward == 0:
                    #then means either didnt reach goal or fell in hole... boooooo
                    didnt_reach_goal_sarsa += 1
                    
                break #eps ends when are in terminal state so break
                
    #get results from running SARSA algor out
    get_results(steps_taken_sarsa, reached_goal_sarsa, didnt_reach_goal_sarsa)

## Results from running SARSA:

In [64]:
#run SARSA algorithm to get results
run_sarsa_algor(env, num_eps, max_steps, gamma, alph, epsilon)

The goal state was reached in an average of 7.034659557013946 steps.
The goal state was reached 97.52 percent of the time.
The goal state was not reached 2.48 percent of the time, either from ending the episode in a frozen state or from falling into a hole.
